In [1]:
import pyshacl
from rdflib import OWL, RDF, RDFS, SH, Graph, Namespace

# running in si-builder venv

In [2]:
S223 = Namespace("http://data.ashrae.org/standard223#")

In [3]:
g = Graph()

g.parse('MODEL_SP223_core-v1.0.ttl', format = 'ttl')

g.parse('VOCAB_SP223_enumeration-v1.0.ttl', format = 'ttl')
g.parse('MODEL_SP223_equipment-v1.0.ttl', format = 'ttl')
g.parse('EQUIPMENTS_EXT_SP223.ttl', format = 'ttl')



g.parse('G36_SP223-v1.0_short.ttl', format = 'ttl')



g.parse('G36_dataGraph_VAV4-2.ttl', format = 'ttl')







og = Graph()
pyshacl.rdfutil.clone.clone_graph(g, og)
sg = Graph()
pyshacl.rdfutil.clone.clone_graph(g, sg)



valid, report_graph, report_text = pyshacl.validate(
    data_graph=g,
    shacl_graph=g,
    advanced=True,
    js=True,
    allow_warnings=False,
    inplace=True,
    iterate_rules=True, # If you iterate rules, this SPARQL construct for chain_rule won't work
)

# 'ex:Generator' in report_text
# print(report_text)

# find the prefix definitions so the select can find them
namespace_map = {}
for prefix, uriref in report_graph.namespaces():
    namespace_map[prefix] = Namespace(uriref)
if "sh" not in namespace_map:
    namespace_map["sh"] = SH

# find the validation results
# Currently filtering out info in qs
qs = """
    SELECT ?resultSeverity ?sourceShape ?resultMessage ?focusNode ?value
    WHERE {
        ?report rdf:type sh:ValidationReport .
        ?report sh:result ?result .
        ?result sh:focusNode ?focusNode .
        OPTIONAL { ?result sh:resultMessage ?resultMessage } .
        ?result sh:resultSeverity ?resultSeverity .
        ?result sh:sourceShape ?sourceShape .
        OPTIONAL { ?result sh:value ?value } .
        # FILTER(?resultSeverity = s223:g36).
        }
    """

# pretty colors
color_map = {SH.Violation: 33, SH.Info: 34, SH.Warning: 35, S223.g36: 36}

# run the query, sort the results
results = sorted(report_graph.query(qs, initNs=namespace_map))

prev = None
for resultSeverity, sourceShape, resultMessage, focusNode, value in results:
    if sourceShape != prev:
        color = color_map[resultSeverity]
        if resultMessage:
            print(f"\x1b[{color}m{resultMessage}\x1b[0m")
        else:
            print(f"\x1b[{color}m{sourceShape}\x1b[0m")
        prev = sourceShape
    print(f"    {focusNode}{' ' + value if value else ''}")


/Users/huangwp/opt/anaconda3/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: rkzeug is an invalid version and will not be supported in a future release
  warnings.warn(


In [4]:
added_triples = g - og
added_triples.print()

In [5]:
print(report_text)

Validation Report
Conforms: True



# Start BuildingMOTIF

In [6]:
# import necessary BuildingMOTIF libraries
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BRICK # import this to make writing URIs easier


In [7]:
# Create an in-memory BuildingMOTIF instance
bm = BuildingMOTIF("sqlite://")

# create the namespace for the building
BLDG = Namespace('urn:bldg/')


# create the building model and add a description
model = Model.create(BLDG, description="This is a test model for a simple building")

# Load data graphs and shape graphs in BuildingMOTIF

In [8]:
# load the BRICK model you created
model.graph.parse("G36_dataGraph_VAV4-2.ttl", format="ttl")

<Graph identifier=1058b5b0-9cff-4d05-b9b6-480661364827 (<class 'rdflib.graph.Graph'>)>

In [9]:
s223_shape_graph = Library.load(ontology_graph="223p.ttl",overwrite=False)


In [10]:
#constraint1 = Library.load(ontology_graph="../../223standard/models/MODEL_SP223_core-v1.0.ttl")
#constraint2 = Library.load(ontology_graph="../../223standard/vocab/VOCAB_SP223_enumeration-v1.0.ttl")
#constraint3 = Library.load(ontology_graph="../../223standard/models/MODEL_SP223_equipment-v1.0.ttl")
#constraint4 = Library.load(ontology_graph="../../223standard/extensions/EQUIPMENTS_EXT_SP223.ttl")

constraint1 = Library.load(ontology_graph="MODEL_SP223_core-v1.0.ttl")
constraint2 = Library.load(ontology_graph="VOCAB_SP223_enumeration-v1.0.ttl")
constraint3 = Library.load(ontology_graph="MODEL_SP223_equipment-v1.0.ttl")
constraint4 = Library.load(ontology_graph="EQUIPMENTS_EXT_SP223.ttl")

In [11]:

constraint_shape = Library.load(ontology_graph="G36_SP223-v1.0_short.ttl")


In [12]:
# load the ".yaml" templates from a folder named "my-templates" into the variable "lib"
lib = Library.load(directory="s223-templates",overwrite=False)

In [13]:
# gather all the ".ttl" constraints shape graphs in one place
all_shape_graphs = [
    s223_shape_graph.get_shape_collection(),
    constraint1.get_shape_collection(),
    constraint2.get_shape_collection(),
    constraint3.get_shape_collection(),
    constraint4.get_shape_collection(),
    constraint_shape.get_shape_collection(),
    lib.get_shape_collection()
]

In [14]:
# view the content of the BRICK model
print(model.graph.serialize())

@prefix ex: <http://example.org#> .
@prefix g36: <http://data.ashrae.org/standard223/1.0/extension/g36#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix s223: <http://data.ashrae.org/standard223#> .
@prefix unit: <http://qudt.org/vocab/unit/> .

g36: a owl:Ontology ;
    owl:imports <http://data.ashrae.org/standard223/1.0/collections/MODEL_SP223_all-v1.0.ttl>,
        <http://data.ashrae.org/standard223/1.0/extensions/G36-rules_SP223-v1.0.shapes.ttl>,
        <http://data.ashrae.org/standard223/1.0/model/all> .

ex:AirFlowStation42 a g36:AirFlowStation ;
    rdfs:label "Supply Airflow Sensor" ;
    s223:hasMeasurementLocation ex:DamperInlet42 ;
    s223:measuresSubstance s223:Medium-Air ;
    s223:observes ex:SupplyAirflow42 .

ex:TemperatureSensor a g36

# Validate with BuildingMOTIF

In [15]:
validation_result = model.validate(all_shape_graphs)
print(f"Model is valid? {validation_result.valid}")
print("-"*79) # just a separator for better error display
print(validation_result.report_string)
print("-"*79) # just a separator for better error display
print("Model is invalid for these reasons:")
for diff in validation_result.diffset:
    print(f" - {diff.reason()}")

Model is valid? False
-------------------------------------------------------------------------------
Validation Report
Conforms: False
Results (18):
Validation Result in ClassConstraintComponent (http://www.w3.org/ns/shacl#ClassConstraintComponent):
	Severity: <http://data.ashrae.org/standard223#g36>
	Source Shape: [ sh:class <http://data.ashrae.org/standard223/1.0/extension/g36#HotWaterValve> ; sh:comment Literal("Must have Valve Connected.") ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:path [ sh:alternativePath ( <http://data.ashrae.org/standard223#connectedFrom> <http://data.ashrae.org/standard223#connectedTo> ) ] ; sh:severity <http://data.ashrae.org/standard223#g36> ]
	Focus Node: ex:HotWaterCoil
	Value Node: ex:Damper42
	Result Path: [ sh:alternativePath ( <http://data.ashrae.org/standard223#connectedFrom> <http://data.ashrae.org/standard223#connectedTo> ) ]
	Message: Value does not have class <http://data.ashrae.org/standard223/1.0/extension/g36#HotWaterValve>
Constrai

 - http://example.org#Space1 needs between 1 and None uses of path nb66f63e3a155446eb189e3cab917d774b103
 - http://example.org#HotWaterCoil needs to be a http://data.ashrae.org/standard223/1.0/extension/g36#HotWaterValve
 - http://example.org#AirFlowStation42 needs between 1 and 1 instances of http://data.ashrae.org/standard223#ObservableProperty on path http://data.ashrae.org/standard223#observesProperty
 - http://example.org#WindowSwitch needs between 1 and 1 instances of http://data.ashrae.org/standard223#ObservableProperty on path http://data.ashrae.org/standard223#observesProperty
 - http://example.org#TemperatureSensor needs between 1 and 1 instances of http://data.ashrae.org/standard223#ObservableProperty on path http://data.ashrae.org/standard223#observesProperty


# Generate Templates

In [16]:
# We first create a new template library by adding any name you like to the new template library. 
# This new template library is stored in the "generated_templates" variable
generated_templates = Library.create("any_template_library_name_you_like")

In [17]:
# We then use the following code to auto-generate ".ttl" templates and add them to the "generated_templates" variable. 
# For each failed validation item "diff", one template is auto-generated
for diff in validation_result.diffset:
    try:
        diff.resolve(generated_templates)
        print("--------iteration---------------")
    except:
        pass

--------iteration---------------
--------iteration---------------
--------iteration---------------
--------iteration---------------


In [18]:
# Print the auto-generated templates. For each auto-generated template, the template name, the template parameters, and the template body are printed. 
for template in generated_templates.get_templates():
    template = template.inline_dependencies()
    print(f"Name (autogenerated): {template.name}")
    print(f"Parameters (autogenerated): {template.parameters}")
    print("Template body (autogenerated):")
    print(template.body.serialize())
    print("-"*79) # just a separator for better display

Name (autogenerated): resolve97d09f33
Parameters (autogenerated): set()
Template body (autogenerated):


-------------------------------------------------------------------------------
Name (autogenerated): resolveb1d91094
Parameters (autogenerated): set()
Template body (autogenerated):
@prefix ex: <http://example.org#> .
@prefix g36: <http://data.ashrae.org/standard223/1.0/extension/g36#> .

ex:HotWaterCoil a g36:HotWaterValve .


-------------------------------------------------------------------------------
Name (autogenerated): resolve_c08f4352
Parameters (autogenerated): {'p2'}
Template body (autogenerated):
@prefix ex: <http://example.org#> .
@prefix s223: <http://data.ashrae.org/standard223#> .

ex:AirFlowStation42 s223:observesProperty <urn:___param___#p2> .

<urn:___param___#p2> a s223:ObservableProperty .


-------------------------------------------------------------------------------
Name (autogenerated): resolve_c5709d02
Parameters (autogenerated): {'p3'}
Template body (au